<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Setup" data-toc-modified-id="Setup-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Setup</a></span><ul class="toc-item"><li><span><a href="#Helper-functions" data-toc-modified-id="Helper-functions-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Helper functions</a></span></li></ul></li><li><span><a href="#Import-data" data-toc-modified-id="Import-data-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Import data</a></span></li><li><span><a href="#Effect-estimation" data-toc-modified-id="Effect-estimation-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Effect estimation</a></span><ul class="toc-item"><li><span><a href="#Plot-raw-timeseries" data-toc-modified-id="Plot-raw-timeseries-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>Plot raw timeseries</a></span></li><li><span><a href="#ITT-on-panelists-who-consume-SSBs" data-toc-modified-id="ITT-on-panelists-who-consume-SSBs-3.2"><span class="toc-item-num">3.2&nbsp;&nbsp;</span>ITT on panelists who consume SSBs</a></span></li><li><span><a href="#Summary-stats-by-soda-percentile" data-toc-modified-id="Summary-stats-by-soda-percentile-3.3"><span class="toc-item-num">3.3&nbsp;&nbsp;</span>Summary stats by soda percentile</a></span></li></ul></li><li><span><a href="#Cook-County" data-toc-modified-id="Cook-County-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Cook County</a></span><ul class="toc-item"><li><span><a href="#Plot-raw-data" data-toc-modified-id="Plot-raw-data-4.1"><span class="toc-item-num">4.1&nbsp;&nbsp;</span>Plot raw data</a></span></li><li><span><a href="#Effects-on-nutrients---monthly" data-toc-modified-id="Effects-on-nutrients---monthly-4.2"><span class="toc-item-num">4.2&nbsp;&nbsp;</span>Effects on nutrients - monthly</a></span><ul class="toc-item"><li><span><a href="#Baseline-DiD" data-toc-modified-id="Baseline-DiD-4.2.1"><span class="toc-item-num">4.2.1&nbsp;&nbsp;</span>Baseline DiD</a></span></li><li><span><a href="#Matching" data-toc-modified-id="Matching-4.2.2"><span class="toc-item-num">4.2.2&nbsp;&nbsp;</span>Matching</a></span></li><li><span><a href="#Synthetic-control" data-toc-modified-id="Synthetic-control-4.2.3"><span class="toc-item-num">4.2.3&nbsp;&nbsp;</span>Synthetic control</a></span></li></ul></li><li><span><a href="#Pooled-time-series" data-toc-modified-id="Pooled-time-series-4.3"><span class="toc-item-num">4.3&nbsp;&nbsp;</span>Pooled time series</a></span><ul class="toc-item"><li><span><a href="#Behaviors" data-toc-modified-id="Behaviors-4.3.1"><span class="toc-item-num">4.3.1&nbsp;&nbsp;</span>Behaviors</a></span></li><li><span><a href="#Nutrients" data-toc-modified-id="Nutrients-4.3.2"><span class="toc-item-num">4.3.2&nbsp;&nbsp;</span>Nutrients</a></span></li><li><span><a href="#Beverages" data-toc-modified-id="Beverages-4.3.3"><span class="toc-item-num">4.3.3&nbsp;&nbsp;</span>Beverages</a></span></li><li><span><a href="#Source-of-sugar" data-toc-modified-id="Source-of-sugar-4.3.4"><span class="toc-item-num">4.3.4&nbsp;&nbsp;</span>Source of sugar</a></span></li><li><span><a href="#Location-of-store" data-toc-modified-id="Location-of-store-4.3.5"><span class="toc-item-num">4.3.5&nbsp;&nbsp;</span>Location of store</a></span></li></ul></li><li><span><a href="#Effects-by-geography" data-toc-modified-id="Effects-by-geography-4.4"><span class="toc-item-num">4.4&nbsp;&nbsp;</span>Effects by geography</a></span></li><li><span><a href="#Using-other-cities-as-controls" data-toc-modified-id="Using-other-cities-as-controls-4.5"><span class="toc-item-num">4.5&nbsp;&nbsp;</span>Using other cities as controls</a></span></li></ul></li><li><span><a href="#Additional-plots" data-toc-modified-id="Additional-plots-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Additional plots</a></span><ul class="toc-item"><li><span><a href="#Cook-County-map" data-toc-modified-id="Cook-County-map-5.1"><span class="toc-item-num">5.1&nbsp;&nbsp;</span>Cook County map</a></span></li><li><span><a href="#Google-Trends" data-toc-modified-id="Google-Trends-5.2"><span class="toc-item-num">5.2&nbsp;&nbsp;</span>Google Trends</a></span></li></ul></li></ul></div>

# SSB Tax Analysis
By Zachary A. Goodman

Updated: 2021-04

This script performs the analysis in Goodman & Orchard (2021).

## Setup

In [ ]:
clear all

* set working directory
qui cd "../"

* set global directories
global raw_data ./data/raw_data
global temp_data ./data/temp_data
global gen_data ./data/gen_data
global figures ./tex/figures 
global tables ./tex/tables

* check dependencies
which cem
which reghdfe

* plot style
set scheme s1color

### Helper functions

In [ ]:
capture program drop balance_panel
program balance_panel
    /*
    Keeps panelists who are present in panel for all months.
    */
    bys household_code: gen N = _N
    qui sum N
    keep if N == r(max)
    end

In [ ]:
capture program drop top_code
program top_code
    /* 
    Top codes >99th percentile of `var' at the 99th percentile
     to avoid the overreaching effects of outliers.
    */
    args var
    qui sum `var', detail
    replace `var' = r(p99) if `var' > r(p99)
    end

capture program drop top_code_all
program top_code_all
    /* 
    Top codes all nutrient vars.
    */
    foreach v of varlist calories-transfatgrams {
        capture top_code `v'
    }
    end

In [ ]:
* define functions for each of four main samples

capture program drop get_sea_sf get_cook get_boulder_oak get_philly

program get_sea_sf
    keep if inlist(dma_cd, 819, 807) & inlist(locality, "Control", "San Francisco", "Seattle")
    end

program get_cook
    keep if dma_cd == 602
    end

program get_boulder_oak
    keep if inlist(dma_cd, 751, 807) & inlist(locality, "Control", "Boulder", "Oakland")
    end

program get_philly
    keep if dma_cd == 504
    end


capture program drop get_cook_data
program get_cook_data

    * get data
    use $gen_data/panelist_nutrition_month_prepped, clear
    get_cook
    // keep if yearmonth < 2019 
    top_code_all

    * add months_since_treated for control units
    bys yearmonth: ereplace months_since_treat = min(months_since_treat)

    end

In [ ]:
* function for replace months_since_treat for control units

capture program drop assign_months_since_treat
program assign_months_since_treat
    replace months_since_treat = . if !evertreated
    bys dma_cd yearmonth: ereplace months_since_treat = min(months_since_treat)
    end

In [ ]:
* function for exporting plots

capture program drop export_img
program export_img
    args name
    graph export $figures/`name'.pdf, name(`name') replace
    end

## Import data

In [ ]:
use $gen_data/panelist_nutrition_month_prepped.dta, clear
top_code_all

## Effect estimation

### Plot raw timeseries

In [ ]:
* plot trends for nutrients

capture restore
preserve

keep if months_since_treat >=-12 & months_since_treat <= 12
collapse (mean) calories-transfatgrams, by(months_since_treat)

sort months_since_treat

foreach v of varlist calories-transfatgrams {
    twoway line `v' months_since_treat, ///
    title("`v'")
}

restore

In [ ]:
* plot trends for control

capture restore
preserve

keep if locality == "Control"
collapse (mean) calories-transfatgrams, by(yearmonth)

sort yearmonth

foreach v of varlist calories-transfatgrams {
    twoway line `v' yearmonth, ///
    title("`v'")
}

restore

### ITT on panelists who consume SSBs

In [ ]:
use $gen_data/panelist_nutrition_month_prepped.dta, clear
top_code_all

In [ ]:
* gen indicator for pre-treatment SSB consumption

capture drop pretreat_soda*

// * get control group months since treat
// assign_months_since_treat

* months with >0 pretreatment soda consumption
gen pretreat_soda_months = oz_soda_regular > 0 & inrange(months_since_treat, -12, -1)
bys household_code: ereplace pretreat_soda_months = sum(pretreat_soda_months)
replace pretreat_soda_months = . if mi(months_since_treat)

* any soda past year
gen pretreat_soda_12mo_any = pretreat_soda_months >= 1
gen pretreat_soda_12mo_half = pretreat_soda_months >= 7

* total oz of soda in year pre treatment
gen pretreat_soda_oz = oz_soda_regular if inrange(months_since_treat, -12, -1)
bys household_code: ereplace pretreat_soda_oz = mean(pretreat_soda_oz)
replace pretreat_soda_oz = . if mi(months_since_treat)

* total oz of soda in year pre treatment, exclude month before tax
gen pretreat_soda_oz_exclude = oz_soda_regular if inrange(months_since_treat, -12, -2)
bys household_code: ereplace pretreat_soda_oz_exclude = mean(pretreat_soda_oz_exclude)
replace pretreat_soda_oz_exclude = . if mi(months_since_treat)

In [ ]:
* flexible program for SSB-percentile interaction estimation

capture program drop get_itt_percentile
program get_itt_percentile
    args locality startmonth endmonth outcome pretreat_var percentiles combo_tile topcode balance weights name treatstart
    /*
    locality: str, where to estimate ITT
    startmonth: int, start of months_since_treat
    outcome: var, outcome var of interest
    endmonth: int, end of months_since_treat
    deciles: int, number of deciles for `pretreat_var'
    combo_tile: int, set percentiles <= combo_tile to 1
    topcode: bool, whether to topcode `outcome'
    balance: bool, whether to balance panel
    weights: bool, whether to include sampling weights
    name: str, name by which to save plot
    treatstart: Optional[int], start month of treatment
    */

    preserve
    
    * get locality subsample and clean
    qui get_`locality'
    qui assign_months_since_treat
    qui keep if inrange(months_since_treat, `startmonth', `endmonth')

    if `topcode' {
        qui top_code `outcome'
    }
    if `balance' {
        qui balance_panel
    }

    * handle treatstart
    drop tau
    gen tau = (months_since_treat >= `treatstart') * evertreated

    * gen percentiles
    capture drop percentiles
    qui egen percentiles = xtile(`pretreat_var') if evertreated, nq(`percentiles')
    replace percentiles = 0 if !evertreated
    tab percentiles
    replace percentiles = 1 if percentiles <= `combo_tile'
    
    * run reg
    if `weights' {
        reghdfe `outcome' c.tau#ib1.percentiles [fw=projection_factor], ///
            absorb(household_code months_since_treat) vce(cluster panelist_zipcd)  
    } 
    else {
        reghdfe `outcome' c.tau#ib1.percentiles, ///
            absorb(household_code months_since_treat) vce(cluster panelist_zipcd)  
    }

    * store results
    eststo `name'

    * fixed effects
    estadd local hhfixed "Yes"
    estadd local mfixed "Yes"
    
    * N
    qui gunique household_code if evertreated
    estadd local treatedhh `r(unique)'
    qui gunique household_code
    estadd local households `r(unique)'
    qui sum household_code
    estadd local household_months `r(N)'

    * balanced panel
    if `balance' {
        estadd local balance "Yes"
    }
    else {
        estadd local balance "No"
    }

    * weights
    if `weights' {
        estadd local weights "Yes"
    }
    else {
        estadd local weights "No"
    }

    restore
    end

In [ ]:
* program for assembling and saving tables

capture program drop export_table
program export_table
    args sto_list name caption note
    esttab `sto_list' using $tables/`name'.tex, cells(b(fmt(3) star) se(fmt(3) par)) ///
    s(treatedhh households household_months hhfixed mfixed balance weights) ///
    drop(_cons) label interaction(" X ") replace  mlabels(none) collabels(none) booktabs ///
    prehead("\begin{spacing}{1.0} \begin{table} \centering \caption{`caption'} \label{`name'} \begin{threeparttable} \begin{tabular}{m{0.23\linewidth}*{@E}{>{\centering\arraybackslash}m{0.10\linewidth}}} \toprule") ///
    postfoot("\bottomrule \end{tabular} \Fignote{`note'} \end{threeparttable} \end{table} \end{spacing}" )
    end

In [ ]:
* program for running the above two programs

capture program drop get_itt_percentile_cook
program get_itt_percentile_cook
    args outcome outcome_text name

    capture drop logoutcome
    gen logoutcome = asinh(`outcome')
    
    eststo clear
    qui get_itt_percentile "cook" -12 3 logoutcome pretreat_soda_oz_exclude 10 2 1 0 0 m1 0
    qui get_itt_percentile "cook" -12 3 logoutcome pretreat_soda_oz_exclude 10 2 1 0 1 m2 0
    qui get_itt_percentile "cook" -12 3 logoutcome pretreat_soda_oz_exclude 10 2 1 1 1 m3 0
    qui get_itt_percentile "cook" -12 7 logoutcome pretreat_soda_oz_exclude 10 2 1 0 0 m4 4
    qui get_itt_percentile "cook" -12 7 logoutcome pretreat_soda_oz_exclude 10 2 1 0 1 m5 4
    qui get_itt_percentile "cook" -12 7 logoutcome pretreat_soda_oz_exclude 10 2 1 1 1 m6 4

    * export to table
    * args sto_list name caption note

    export_table m* `name' ///
        "Effects of tax on `outcome_text' by pretreatment soda decile, Cook County" ///
        "Coefficients represent the percent change in `outcome_text' purchased during the four months that the Cook County beverage tax was active (1 - 3) and the four months after the tax was no longer active (4 - 6), separated by pretreatment regular soda volume purchase decile. Deciles were calculated based on the treated households' average monthly volume of soda purchased during the previous year omitting the month prior to the tax. All of decile 1 and nearly all of decile 2 report zero regular soda purchases before the tax. \Regnote"

    end

In [ ]:
* Cook, outcome = g of sugar
get_itt_percentile_cook sugargrams "sugar" sodatilesgcook

In [ ]:
* Cook, outcome = ounces of regular soda
get_itt_percentile_cook oz_soda_regular "regular soda volume" sodatilesozcook

In [ ]:
* Seattle and San Fran, outcome = g sugar

* args locality startmonth endmonth outcome pretreat_var percentiles combo_tile topcode balance weights name

capture drop logsugargrams
gen logsugargrams = asinh(sugargrams)

eststo clear
qui get_itt_percentile "sea_sf" -12 3 logsugargrams pretreat_soda_oz_exclude 5 2 1 0 0 m1 0
qui get_itt_percentile "sea_sf" -12 3 logsugargrams pretreat_soda_oz_exclude 5 2 1 0 1 m2 0
qui get_itt_percentile "sea_sf" -12 3 logsugargrams pretreat_soda_oz_exclude 5 2 1 1 1 m3 0
qui get_itt_percentile "sea_sf" -12 12 logsugargrams pretreat_soda_oz_exclude 5 2 1 0 0 m4 0
qui get_itt_percentile "sea_sf" -12 12 logsugargrams pretreat_soda_oz_exclude 5 2 1 0 1 m5 0
qui get_itt_percentile "sea_sf" -12 12 logsugargrams pretreat_soda_oz_exclude 5 2 1 1 1 m6 0

export_table m* sodatilesgsf ///
    "Effects of tax on sugar by pretreatment soda quintile, Seattle and San Francisco" ///
    "Coefficients represent the percent change in sugar purchased during the first four months (1 - 3) and year (4 - 6) after the Seattle and San Francisco beverage taxes became active, separated by pretreatment regular soda volume purchase decile. Quintiles were calculated based on the treated households' average monthly volume of soda purchased during the previous year omitting the month prior to the tax. All of quintile 1 and nearly all of quintile 2 report zero regular soda purchases before the tax. \Regnote"

In [ ]:
* Philly, outcome = g sugar

* args locality startmonth endmonth outcome pretreat_var percentiles combo_tile topcode balance weights name

eststo clear
qui get_itt_percentile "philly" -12 3 logsugargrams pretreat_soda_oz_exclude 5 1 1 0 0 m1 0
qui get_itt_percentile "philly" -12 3 logsugargrams pretreat_soda_oz_exclude 5 1 1 0 1 m2 0
qui get_itt_percentile "philly" -12 3 logsugargrams pretreat_soda_oz_exclude 5 1 1 1 1 m3 0
qui get_itt_percentile "philly" -12 12 logsugargrams pretreat_soda_oz_exclude 5 1 1 0 0 m4 0
qui get_itt_percentile "philly" -12 12 logsugargrams pretreat_soda_oz_exclude 5 1 1 0 1 m5 0
qui get_itt_percentile "philly" -12 12 logsugargrams pretreat_soda_oz_exclude 5 1 1 1 1 m6 0

export_table m* sodatilesgphilly ///
    "Effects of tax on sugar by pretreatment soda quintile, Philadelphia" ///
    "Coefficients represent the percent change in sugar purchased during the first four months (1 - 3) and year (4 - 6) after the Philadelphia beverage tax became active, separated by pretreatment regular soda volume purchase decile. Quintiles were calculated based on the treated households' average monthly volume of soda purchased during the previous year omitting the month prior to the tax. Nearly all of quintile 1 report zero regular soda purchases before the tax. \Regnote"

In [ ]:
* Boulder and Oakland, outcome = g sugar

* args locality startmonth endmonth outcome pretreat_var percentiles combo_tile topcode balance weights name

eststo clear
qui get_itt_percentile "boulder_oak" -12 3 logsugargrams pretreat_soda_oz_exclude 5 1 1 0 0 m1 0
qui get_itt_percentile "boulder_oak" -12 3 logsugargrams pretreat_soda_oz_exclude 5 1 1 0 1 m2 0
qui get_itt_percentile "boulder_oak" -12 3 logsugargrams pretreat_soda_oz_exclude 5 1 1 1 1 m3 0
qui get_itt_percentile "boulder_oak" -12 12 logsugargrams pretreat_soda_oz_exclude 5 1 1 0 0 m4 0
qui get_itt_percentile "boulder_oak" -12 12 logsugargrams pretreat_soda_oz_exclude 5 1 1 0 1 m5 0
qui get_itt_percentile "boulder_oak" -12 12 logsugargrams pretreat_soda_oz_exclude 5 1 1 1 1 m6 0

export_table m* sodatilesgoak ///
    "Effects of tax on sugar by pretreatment soda quintile, Boulder and Oakland" ///
    "Coefficients represent the percent change in sugar purchased during the first four months (1 - 3) and year (4 - 6) after the Boulder and Oakland beverage taxes became active, separated by pretreatment regular soda volume purchase decile. Quintiles were calculated based on the treated households' average monthly volume of soda purchased during the previous year omitting the month prior to the tax. All of quintile 1 report zero regular soda purchases before the tax. \Regnote"

In [ ]:
* Triple diff?

### Summary stats by soda percentile

In [ ]:
* TODO

## Cook County

Here we look at Cook Co. by itself given its larger geographical coverage and that it repealed the tax.

In [ ]:
get_cook_data
gunique household_code year month

### Plot raw data

In [ ]:
* Plot function

capture program drop plot_cook
program plot_cook
    /*
    Plots time series of raw data for `var'
    for Cook County vs control.
    */
    args var title plotname ytitle
    twoway (line `var' months_since_treat if evertreated, lcol("cranberry")) ///
        (line `var' months_since_treat if !evertreated, lcol("navy")), ///
        xline(0, lpattern(dash)) xline(3, lpattern(dash)) ///
        title("`title'") ///
        ytitle("`ytitle'") ///
        xtitle("") ///
        name(`plotname', replace) ///
        legend(off) // legend(order(1 "Treatment" 2 "Control")) ///

    end

In [ ]:
* Panelist behaviors

capture restore
preserve

* percent vars
gen p_storebrand = items_storebrand / items_scanned
gen p_food = items_food / items_scanned
gen p_coupon = items_coupons / items_scanned
gen p_deals = items_deals / items_scanned

local vars = "trips total_spent items_deals-coupons_amount_saved p_*"
local titles = "Trips per"
local nvars : word count `vars'

collapse (mean) `vars', by(evertreated months_since_treat)

plot_cook trips "All trips" "trips" "Trips/month"
plot_cook total_spent "All expenditures" "total_spent" "Dollars/month"
plot_cook final_price_paid "Expenditures scanned" "total_scan" "Dollars/month"
plot_cook final_price_paid_food "Food expenditures scanned" "total_scan_food" "Dollars/month"
plot_cook items_scanned "Items scanned" "items_scan" "Items/month"
plot_cook p_food "Percent scanned items food" "p_food" "Percent"
plot_cook p_storebrand "Percent scanned items storebrand" "p_storebrand" "Percent"
plot_cook p_coupon "Percent items coupon used" "p_coupon" "Percent"
plot_cook p_deals "Percent items with deals" "p_deals" "Percent"

graph combine trips total_spent total_scan total_scan_food ///
    items_scan p_food p_storebrand p_coupon p_deals, ///
    row(3) name(panelist_behav, replace) altshrink iscale(1.2) ysize(5) xsize(6)

* export
export_img panelist_behav


* alt method: iterate over two list
// forval i=1/`nvars' {
//     local thisvar : word `i' of `vars'
//     local thistitle : word `i' of `titles'
//     plot_cook `thisvar' `thistitle'
// }

restore

In [ ]:
* Nutrients

capture restore
preserve

* percent vars
gen p_sugar = (sugargrams * 4) / calories

local vars = "calories-sugargrams p_sugar"

collapse (mean) `vars', by(evertreated months_since_treat)

plot_cook calories "Calories per month" "calories" "kcal/month"
plot_cook sugargrams "Sugars per month" "sugar" "g/month"
plot_cook p_sugar "Percent calories from sugar" "p_sugar" "Percent"

plot_cook fatgrams "Fats per month" "fats" "g/month"
plot_cook satfatgrams "Saturated fats per month" "satfats" "g/month"
plot_cook carbsgrams "Carbohydrates per month" "carbs" "g/month"

plot_cook fibergrams "Fiber per month" "fiber"  "g/month"
plot_cook proteingrams "Protein per month" "protein" "g/month"
plot_cook sodiumgrams "Sodium per month" "sodium" "g/month"

graph combine calories sugar p_sugar fats ///
    satfats carbs fiber protein sodium, ///
    row(3) name(panelist_nutr, replace) altshrink iscale(1.2) ysize(5) xsize(6)

* export
export_img panelist_nutr


restore

In [ ]:
* Beverages

capture restore
preserve


local vars = "oz_alcohol-oz_soft_drinks_powder"

collapse (mean) `vars', by(evertreated months_since_treat)

plot_cook oz_soda_regular "Regular soda" "oz_soda_regular" "oz/month"
plot_cook oz_soda_diet "Diet soda" "oz_soda_diet"  "oz/month"
plot_cook oz_fruit_drinks "Fruit drinks" "oz_fruit_drinks" "oz/month"

plot_cook oz_milk "Milk" "oz_milk" "oz/month"
plot_cook oz_water_bottled "Bottled water" "oz_water_bottled" "oz/month"
plot_cook oz_alcohol "Alcohol" "oz_alcohol" "oz/month"

plot_cook oz_tea_liquid "Liquid tea" "oz_tea_liquid" "oz/month"
plot_cook oz_coffee_fluid "Liquid coffee" "oz_coffee_fluid" "oz/month"
plot_cook oz_coffee_solid "Solid coffee" "oz_coffee_solid" "oz/month"

graph combine oz_soda_regular oz_soda_diet oz_fruit_drinks ///
    oz_milk oz_water_bottled oz_alcohol ///
    oz_tea_liquid oz_coffee_fluid oz_coffee_solid, ///
    row(3) name(panelist_bev, replace) altshrink iscale(1.2) ysize(5) xsize(6)

* export
export_img panelist_bev


restore

In [ ]:
* Sugar from sources

capture restore
preserve

* gen vars
gen sugar_beverages = sugargrams - sugar_nonbeverage

* percent vars
count if sugargrams == 0
gen sugar_beverages_p = sugar_beverages / sugargrams

local vars = "sugar_* sugargrams"

collapse (mean) `vars', by(evertreated months_since_treat)

plot_cook sugargrams "All sugar" "sugar" "g/month"
plot_cook sugar_beverages "Sugar from beverages" "sugar_beverages" "g/month"
plot_cook sugar_nonbeverage "Sugar from non-beverages" "sugar_nonbeverage" "g/month"
plot_cook sugar_beverages_p "Percent sugar from beverages" "sugar_beverages_p" "Percent"

plot_cook sugar_soda_regular "Sugar from regular soda" "sugar_soda_regular" "g/month"
plot_cook sugar_soda_diet "Sugar from diet soda" "sugar_soda_diet" "g/month"
plot_cook sugar_milk "Sugar from milk" "sugar_milk" "g/month"

plot_cook sugar_fruit_punch "Sugar from fruit punch" "sugar_fruit_punch" "g/month"
// TODO: plot_cook sugar_fruit_drinks "Sugar from fruit drinks" "sugar_fruit_drinks" "g/month"
plot_cook sugar_coffee_fluid "Sugar from coffee drinks" "sugar_coffee_fluid" "g/month"
plot_cook sugar_tea_liquid "Sugar from tea drinks" "sugar_tea_liquid" "g/month"

plot_cook sugar_candy "Sugar from candy" "sugar_candy" "g/month"
plot_cook sugar_cookies "Sugar from cookies" "sugar_cookies" "g/month"
plot_cook sugar_storebrand "Sugar from storebrand products" "sugar_storebrand" "g/month"


graph combine sugar sugar_beverages sugar_nonbeverage ///
    sugar_beverages_p sugar_soda_regular sugar_milk ///
    sugar_fruit_punch sugar_coffee_fluid sugar_tea_liquid ///
    sugar_candy sugar_cookies sugar_storebrand, ///
    row(4) name(panelist_sugar_sources, replace) altshrink iscale(1.2) ysize(7) xsize(6)

* export
export_img panelist_sugar_sources


restore

In [ ]:
* Cook stores

capture restore
preserve

* replace cook vars for control group (for plotting)
replace cook_trip = trips - cook_trip if !evertreated
replace cook_dollars = total_spent - cook_dollars if !evertreated
replace cook_soda_regular = oz_soda_regular - cook_soda_regular if !evertreated
replace cook_soda_diet = oz_soda_diet - cook_soda_diet if !evertreated
replace cook_alcohol = oz_alcohol - cook_alcohol if !evertreated
replace cook_sugar = sugargrams - cook_sugar if !evertreated
replace cook_sugar_soda_regular = sugar_soda_regular - cook_sugar_soda_regular if !evertreated
replace cook_sugar_nonbeverage = sugar_nonbeverage - cook_sugar_nonbeverage if !evertreated
gen cook_sugar_beverage = cook_sugar - cook_sugar_nonbeverage

* gen percent vars
gen cook_trips_p = cook_trip / trips
gen cook_dollars_p = cook_dollars / total_spent
gen cook_soda_regular_p = cook_soda_regular / oz_soda_regular
gen cook_soda_diet_p = cook_soda_diet / oz_soda_diet
gen cook_sugar_p = cook_sugar / sugargrams
gen cook_sugar_soda_regular_p = cook_sugar_soda_regular / sugar_soda_regular

* gen inverse vars
gen cook_trip_inv = trips - cook_trip
gen cook_dollars_inv = total_spent - cook_dollars
gen cook_soda_regular_inv = oz_soda_regular - cook_soda_regular
gen cook_soda_diet_inv = oz_soda_diet - cook_soda_diet
gen cook_sugar_inv = sugargrams - cook_sugar
gen cook_sugar_soda_regular_inv = sugar_soda_regular - cook_sugar_soda_regular
gen cook_sugar_beverage_inv = (sugargrams - sugar_nonbeverage) - cook_sugar_beverage
gen cook_sugar_nonbeverage_inv = sugar_nonbeverage - cook_sugar_nonbeverage

local vars = "cook_*"

collapse (mean) `vars', by(evertreated months_since_treat)

local where = "stores in locality"
local wherenot = "stores outside locality"

plot_cook cook_trip "Trips, `where'" "cook_trip" "trips/month"
plot_cook cook_trip_inv "Trips, `wherenot'" "cook_trip_inv" "trips/month"
plot_cook cook_dollars "Total spent, `where'" "cook_dollars" "dollars/month"
plot_cook cook_dollars_inv "Total spent, `wherenot'" "cook_dollars_inv" "dollars/month"

// plot_cook cook_soda_regular "Regular soda purchased at `where'" "cook_soda_regular" "oz/month"
// plot_cook cook_soda_diet "Diet soda purchased at `where'" "cook_soda_diet" "oz_month"
// plot_cook cook_soda_regular_p "Percent regular soda purchased at `where'" "cook_soda_regular_p" "Percent"
// plot_cook cook_soda_diet_p "Percent diet soda purchased at `where'" "cook_soda_diet_p" "Percent"
// plot_cook cook_soda_regular_inv "Regular soda purchased at `wherenot'" "cook_soda_inv" "oz_month"
// plot_cook cook_soda_diet_inv "Diet soda purchased at `wherenot'" "cook_diet_inv" "oz_month"

// plot_cook cook_sugar "Sugar purchased at `where'" "cook_sugar" "g/month"
// plot_cook cook_sugar_p "Percent sugar purchased at `where'" "cook_sugar_p" "Percent"
// plot_cook cook_sugar_inv "Sugar purchased at `wherenot'" "cook_sugar_inv" "g/month"
// plot_cook cook_sugar_soda_regular "Soda sugar purchased at `where'" "cook_sugar_soda_regular" "g/month"
// plot_cook cook_sugar_soda_regular_p "Percent soda sugar purchased at `where'" "cook_sugar_soda_regular_p" "Percent"
// plot_cook cook_sugar_soda_regular_inv "Soda sugar purchased at `wherenot'" "cook_sugar_soda_regular_inv" "g/month"

plot_cook cook_sugar_beverage "Bev. sugar, `where'" "cook_sugar_beverage" "g/month"
plot_cook cook_sugar_beverage_inv "Bev. sugar, `wherenot'" "cook_sugar_beverage_inv" "g/month"
plot_cook cook_sugar_nonbeverage "Non-bev. sugar, `where'" "cook_sugar_nonbeverage" "g/month"
plot_cook cook_sugar_nonbeverage_inv "Non-bev. sugar, `wherenot'" "cook_sugar_nonbeverage_inv" "g/month"


graph combine cook_trip cook_trip_inv ///
    cook_dollars cook_dollars_inv ///
    cook_sugar_beverage cook_sugar_beverage_inv ///
    cook_sugar_nonbeverage cook_sugar_nonbeverage_inv, ///
    row(4) name(panelist_cook_stores, replace) altshrink iscale(1.2) ysize(7) xsize(6)

* export
export_img panelist_cook_stores


restore

### Effects on nutrients - monthly

#### Baseline DiD

In [ ]:
* Define function

capture program drop reg_nutr
program reg_nutr
    args nutr nutrname

    * gen months var > 0
    capture drop temp_months
    qui sum months_since_treat
    local min_months = r(min)
    local base = -3 - `min_months'
    gen temp_months = months_since_treat - `min_months'

    * gen treatment var if missing
    capture gen treated = locality == "cook"

    * run regression
    capture drop log_`nutr'
    gen log_`nutr' = asinh(`nutr')
    reghdfe log_`nutr' evertreated c.evertreated#ib`base'.temp_months, ///
        absorb(household_code temp_months) vce(cluster panelist_zipcd)
    drop temp_months log_`nutr'

    * coeffplot labels
    local labels = ""
    local keeplist = ""
    forvalues i = 7/34 {
        local j = `i' + `min_months'
        local labels = "`labels'" + `"`i'.* = `j' "'
        local keeplist = "`keeplist'" + `"`i'.* "'
    }

    * coeffplot
    local title = proper("`nutrname'")
    local start_policy = 12.5
    local end_policy = `start_policy' + 4
    coefplot, mcol("navy") ciopts(lcol("navy")) keep(`keeplist') vertical ///
        yline(0, lcol(black)) xline(`start_policy', lpattern(dash)) ///
        xline(`end_policy', lpattern(dash)) ///
        baselevels omitted nolabel coeflabels(`labels') ///
        xtitle("Months since tax began") ///
        ytitle("Percent change in `nutrname' purchased") ///
        title("`title'") ///
        ylabel(-0.2(0.1)0.2) ///
        name(coefplot_cook_`nutr', replace)

    end

In [ ]:
reg_nutr sugargrams "sugar"
reg_nutr calories "calories"
reg_nutr fatgrams "fat"
reg_nutr carbsgrams "carbohydrates"
reg_nutr proteingrams "protein"
reg_nutr fibergrams "fiber"

graph combine coefplot_cook_calories coefplot_cook_sugargrams ///
    coefplot_cook_carbsgrams coefplot_cook_fibergrams ///
    coefplot_cook_fatgrams coefplot_cook_proteingrams, ///
    row(3) name(coefplot_cook_combo, replace) ///
    altshrink iscale(1.2) ysize(5) xsize(6)

export_img coefplot_cook_combo

In [ ]:
reg_nutr oz_soda_regular "regular soda"
reg_nutr oz_soda_diet "diet soda"
reg_nutr oz_alcohol "alcohol"
reg_nutr oz_water_bottled "water"
reg_nutr oz_coffee_fluid "coffee"
reg_nutr oz_milk "milk"

graph combine coefplot_cook_oz_soda_regular coefplot_cook_oz_soda_diet ///
    coefplot_cook_oz_water_bottled coefplot_cook_oz_alcohol ///
    coefplot_cook_oz_coffee_fluid coefplot_cook_oz_milk, ///
    row(3) name(coefplot_cook_bev_combo, replace) ///
    altshrink iscale(1.2) ysize(5) xsize(6)

export_img coefplot_cook_bev_combo

In [ ]:
* TODO sugar from SSBs
* TODO sugar by store location
* TODO heterogeneity by income

#### Matching

In [ ]:
* merge demographic information

merge m:1 household_code year using $gen_data/panelist_demographics.dta, keep(3) nogen

In [ ]:
tab hh_emp_male, nolab

In [ ]:
* get matches via CEM

cem hh_race (1 2 3 4) ///
    hh_size (1 2 3 4 5 6) ///
    hh_income (1 2 3 4) ///
    hh_age (1 2 3 4) ///
    hh_educ (1 2 3 4) ///
    hh_child (0 1) ///
    hh_emp_male (0 1 2 3 9) ///
    hh_emp_female (0 1 2 3 9) if year == 2017, tr(evertreated)

In [ ]:
* estimate trends with matched subset

preserve
bys household_code: ereplace cem_matched = min(cem_matched)
keep if cem_matched

reg_nutr sugargrams "sugar"

restore

#### Synthetic control

In [ ]:
get_cook_data
keep if yearmonth < 2019

* aggregate to zip level
collapse (mean) sugargrams total_spent, by(panelist_zipcd yearmonth months_since_treat evertreated tau)

* balance panel
bys panelist_zipcd: keep if _N == 36

In [ ]:
* aggregate treated units

bys evertreated yearmonth: ereplace sugargrams = mean(sugargrams) if evertreated
bys evertreated yearmonth: ereplace total_spent = mean(total_spent) if evertreated
by evertreated yearmonth: keep if _n == 1 | !evertreated
replace panelist_zipcd = 1 if evertreated
// tab yearmonth evertreated


* set panel 
replace months_since_treat = months_since_treat + 19
tsset panelist_zipcd months_since_treat

In [ ]:
export delimited $temp_data/synth_prepped.csv

In [ ]:
* run synthetic control
synth sugargrams total_spent sugargrams, trunit(1) trperiod(19) ///
    figure keep($gen_data/synth.dta) replace

In [ ]:
use $gen_data/synth.dta, clear

In [ ]:
sum

### Pooled time series

Here we determine average effect during the taxed period and shortly following the taxed period.

In [ ]:
get_cook_data

In [ ]:
* define function for estimating ITT

capture program drop get_itt_cook_pooled
program get_itt_cook_pooled
    args startmonth endmonth treatstart drop_start drop_end outcome topcode balance weights name
    /*
    startmonth: int, start of months_since_treat
    outcome: var, outcome var of interest
    endmonth: int, end of months_since_treat
    treatstart: int, start month of treatment
    drop_start: int, start month to drop (for setting reference months)
    drop_end: int, end month to drop
    topcode: bool, whether to topcode `outcome'
    balance: bool, whether to balance panel
    weights: bool, whether to include sampling weights
    name: str, name by which to save plot
    */

    preserve
    
    * get locality subsample and clean
    qui assign_months_since_treat
    qui keep if inrange(months_since_treat, `startmonth', `endmonth')

    if `topcode' {
        qui top_code `outcome'
    }
    if `balance' {
        qui balance_panel
    }

    * handle treatstart
    drop tau
    gen tau = (months_since_treat >= `treatstart') * evertreated

    * drop months
    drop if inrange(months_since_treat, `drop_start', `drop_end')
    
    * run reg
    if `weights' {
        reghdfe `outcome' tau [fw=projection_factor], ///
            absorb(household_code months_since_treat) vce(cluster panelist_zipcd)  
    } 
    else {
        reghdfe `outcome' tau, ///
            absorb(household_code months_since_treat) vce(cluster panelist_zipcd)  
    }

    * store results
    eststo `name'
    
    * N
    qui gunique household_code if evertreated
    estadd local treatedhh `r(unique)'
    qui gunique household_code
    estadd local households `r(unique)'
    qui sum household_code
    estadd local household_months `r(N)'

    * balanced panel
    if `balance' {
        estadd local balance "Yes"
    }
    else {
        estadd local balance "No"
    }

    * weights
    if `weights' {
        estadd local weights "Yes"
    }
    else {
        estadd local weights "No"
    }

    restore
    end

In [ ]:
* programs for assembling and saving tables

capture program drop export_table_top export_table_middle export_table_bottom

program export_table_top
    args sto_list name outcome_str caption
    esttab `sto_list' using $tables/`name'.tex, cells(b(fmt(3) star) se(fmt(3) par)) ///
    drop(_cons) mlabels(none) collabels(none) noobs fragment ///
    prehead("\begin{spacing}{1.0} \begin{table} \centering \caption{`caption'} \label{`name'} \begin{threeparttable} \begin{tabular}{m{0.23\linewidth}*{@E}{>{\centering\arraybackslash}m{0.10\linewidth}}} \toprule") ///
    rename(tau `outcome_str') replace booktabs substitute(\_ _)
    end

program export_table_middle
    args sto_list name outcome_str
    esttab `sto_list' using $tables/`name'.tex, cells(b(fmt(3) star) se(fmt(3) par)) ///
    drop(_cons) mlabels(none) collabels(none) noobs nonum fragment ///
    rename(tau "`outcome_str'") append booktabs substitute(\_ _)
    end

program export_table_bottom
    args sto_list name outcome_str note
    esttab `sto_list' using $tables/`name'.tex, cells(b(fmt(3) star) se(fmt(3) par)) ///
    s(treatedhh households household_months balance weights) ///
    drop(_cons) mlabels(none) collabels(none) noobs nonum fragment ///
    rename(tau "`outcome_str'") append booktabs substitute(\_ _) ///
    postfoot("\bottomrule \end{tabular} \Fignote{`note'} \end{threeparttable} \end{table} \end{spacing}" )
    end

In [ ]:
* Program for estimating several ITTs and stacking

capture program drop get_itts
program get_itts
    args tablename vars title level

    * start table

    di 1

    capture drop logoutcome
    local var1 : word 1 of `vars'

    if `level' {
        gen logoutcome = `var1'
    }
    else {
        gen logoutcome = asinh(`var1')
    }

    eststo clear
    qui get_itt_cook_pooled -12 3 0 4 4 logoutcome 1 0 0 m1
    qui get_itt_cook_pooled -12 3 0 4 4 logoutcome 1 0 1 m2
    qui get_itt_cook_pooled -12 3 0 4 4 logoutcome 1 1 1 m3
    qui get_itt_cook_pooled -12 7 4 0 3 logoutcome 1 0 0 m4
    qui get_itt_cook_pooled -12 7 4 0 3 logoutcome 1 0 1 m5
    qui get_itt_cook_pooled -12 7 4 0 3 logoutcome 1 1 1 m6
    export_table_top m* `tablename' "`var1'" "`title'"


    * middle table

    local list_length : word count `vars'
    local end = `list_length' - 1

    forvalues i = 2/`end' {
        
        di `i'

        local v : word `i' of `vars'

        * take inv hyperbolic sine of outcome
        capture drop logoutcome
        if `level' {
            gen logoutcome = `v'
        }
        else {
            gen logoutcome = asinh(`v')
        }

        * fit model
        * args startmonth endmonth outcome topcode balance weights name treatstart
        eststo clear
        qui get_itt_cook_pooled -12 3 0 4 4 logoutcome 1 0 0 m1
        qui get_itt_cook_pooled -12 3 0 4 4 logoutcome 1 0 1 m2
        qui get_itt_cook_pooled -12 3 0 4 4 logoutcome 1 1 1 m3
        qui get_itt_cook_pooled -12 7 4 0 3 logoutcome 1 0 0 m4
        qui get_itt_cook_pooled -12 7 4 0 3 logoutcome 1 0 1 m5
        qui get_itt_cook_pooled -12 7 4 0 3 logoutcome 1 1 1 m6

        export_table_middle m* `tablename' "`v'"
    }


    * end table

    di `list_length'

    local varN : word `list_length' of `vars'

    capture drop logoutcome
    if `level' {
        gen logoutcome = `varN'
    }
    else {
        gen logoutcome = asinh(`varN')
    }

    eststo clear
    qui get_itt_cook_pooled -12 3 0 4 4 logoutcome 1 0 0 m1
    qui get_itt_cook_pooled -12 3 0 4 4 logoutcome 1 0 1 m2
    qui get_itt_cook_pooled -12 3 0 4 4 logoutcome 1 1 1 m3
    qui get_itt_cook_pooled -12 7 4 0 3 logoutcome 1 0 0 m4
    qui get_itt_cook_pooled -12 7 4 0 3 logoutcome 1 0 1 m5
    qui get_itt_cook_pooled -12 7 4 0 3 logoutcome 1 1 1 m6
    export_table_bottom m* `tablename' "`varN'" ///
        "Coefficients represent the percent change in the dependent variable during the four months that the Cook County beverage tax was active (1 - 3) and the four months after the tax was no longer active (4 - 6) relative to the 12 months preceding the tax. \FEnote \Regnote"

    end

In [ ]:
/* Outcomes:
Sugar
All sugar
Sugar from soda
Sugar from beverages
Sugar from nonbeverages

Beverages
Regular soda
Diet soda
Alcohol
Water
P(any soda)

Behaviors
All trips
Dollars spent
Trips to treated stores
Dollars at treated stores
*/

#### Behaviors

In [ ]:
* args tablename vars title note

local tablename = "itt_cook_behaviors"
local vars "trips total_spent final_price_paid final_price_paid_food items_scanned items_food items_storebrand items_coupons items_deals"
local title = "Effects of Cook County Beverage Tax on panelist behaviors"
get_itts "`tablename'" "`vars'" "`title'" 0

#### Nutrients

In [ ]:
capture drop calories_nonsugar carbsgrams_nonsugar
gen calories_nonsugar = calories - 4 * sugargrams
gen carbsgrams_nonsugar = carbsgrams - sugargrams

* args tablename vars title note

local tablename = "itt_cook_nutrients"
local vars "sugargrams carbsgrams carbsgrams_nonsugar calories calories_nonsugar fatgrams fibergrams proteingrams sodiumgrams"
local title = "Effects of Cook County Beverage Tax on nutrients"
get_itts "`tablename'" "`vars'" "`title'" 0

#### Beverages

In [ ]:
* args tablename vars title note

local tablename = "itt_cook_beverages"
local vars "oz_soda_regular oz_soda_diet oz_fruit_drinks oz_milk oz_water_bottled oz_alcohol oz_tea_liquid oz_coffee_fluid oz_coffee_solid"
local title = "Effects of Cook County Beverage Tax on select beverage volume"
get_itts "`tablename'" "`vars'" "`title'" 0

In [ ]:
* intensive margin

capture restore
preserve

local vars = "oz_soda_regular oz_soda_diet oz_fruit_drinks oz_milk oz_water_bottled oz_alcohol oz_tea_liquid oz_coffee_fluid oz_coffee_solid"
foreach v of varlist `vars' {
    replace `v' = `v' > 0
}

local tablename = "itt_cook_beverages_intensive"
local title = "Effects of Cook County Beverage Tax on probability of purchasing select beverages"
get_itts "`tablename'" "`vars'" "`title'" 1


restore

#### Source of sugar

In [ ]:
* TODO add sugar_fruit_drinks

capture drop sugar_sweets
gen sugar_sweets = sugar_candy + sugar_cookies

local tablename = "itt_cook_source"
// local vars "sugargrams sugar_nonbeverage sugar_soda_regular sugar_fruit_drinks sugar_milk sugar_tea_liquid sugar_coffee_fluid sugar_candy sugar_cookies"
local vars "sugargrams sugar_soda_regular sugar_milk sugar_tea_liquid sugar_coffee_fluid sugar_sweets"
local title = "Effects of Cook County Beverage Tax on sugar from select sources"
get_itts "`tablename'" "`vars'" "`title'" 1

#### Location of store

In [ ]:
capture restore
preserve

* gen cross border vars
gen cook_trip_inv = trips - cook_trip
gen cook_dollars_inv = total_spent - cook_dollars
gen cook_sugar_inv = sugargrams - cook_sugar

* replace vars for control group
replace cook_trip = trips if !evertreated
replace cook_trip_inv = trips if !evertreated
replace cook_dollars = total_spent if !evertreated
replace cook_dollars_inv = total_spent if !evertreated
replace cook_sugar = sugargrams if !evertreated
replace cook_sugar_inv = sugargrams if !evertreated

* args tablename vars title note

local tablename = "itt_cook_cross_border"
local vars "cook_trip cook_trip_inv cook_dollars cook_dollars_inv cook_sugar cook_sugar_inv"
local title = "Effects of Cook County Beverage Tax on cross-border trips"
get_itts "`tablename'" "`vars'" "`title'" 0

restore

### Effects by geography

In [ ]:
* restore data
get_cook_data
qui do ./jupyter/cook_zip_layers.do

In [ ]:
* estimate effects as function of layer

capture restore
preserve

* restrict years
* Why? Isolate effect of tax during tax from effect post removal
keep if inrange(months_since_treat, -12, 3)

* pretreat soda, for restricting sample
gen pretreat_soda_oz = oz_soda_regular if inrange(months_since_treat, -12, -1)
bys household_code: ereplace pretreat_soda_oz = mean(pretreat_soda_oz)
keep if pretreat_soda_oz > 0
tab layer

* prep outcome var
gen logsugar = asinh(sugargrams)

* drop cross border obs
drop if layer == 1

* combine layers for power
replace layer = 3 if layer == 4
replace layer = 5 if layer == 6
tab layer

* gen and label treatment vars
gen treat1 = (layer == 2) * tau
gen treat2 = (layer == 3) * tau
gen treat3 = (layer == 5) * tau
label var treat1 "Border zips"
label var treat2 "Middle layer"
label var treat3 "Innermost layer"

* gen treatment var if missing
bys yearmonth: ereplace tau = max(tau) if layer > 0

* N
qui gunique household_code if evertreated
local treatedhh = `r(unique)'
qui gunique household_code
local households = `r(unique)'
qui sum household_code
local household_months = `r(N)'

* run regressions
eststo clear
qui reghdfe logsugar treat*, ///
    absorb(household_code yearmonth) vce(cluster panelist_zipcd)
eststo m1
qui estadd local weights "No"
qui estadd local balance "No"
qui estadd local treatedhh `treatedhh'
qui estadd local households `households'
qui estadd local household_months `household_months'


qui reghdfe logsugar treat* [fw=projection_factor], ///
    absorb(household_code yearmonth) vce(cluster panelist_zipcd)
eststo m2
qui estadd local weights "Yes"
qui estadd local balance "No"
qui estadd local treatedhh `treatedhh'
qui estadd local households `households'
qui estadd local household_months `household_months'

balance_panel
qui reghdfe logsugar treat* [fw=projection_factor], ///
    absorb(household_code yearmonth) vce(cluster panelist_zipcd)
eststo m3
qui estadd local weights "Yes"
qui estadd local balance "Yes"
qui gunique household_code if evertreated
qui estadd local treatedhh `r(unique)'
qui gunique household_code
qui estadd local households `r(unique)'
qui sum household_code
qui estadd local household_months `r(N)'

local caption = "Effects of Cook County Beverage Tax by zip code layer"
local name = "itt_geography"
local note = "Coefficients represent the percent change in sugar purchased during the four months that the Cook County beverage tax was active, separated by zip code layer within Cook County. The omitted group is zip codes outside of Cook County, and the specification does not include zip codes that cross the border. The sample includes only households that have positive purchases of regular soda during the year prior to the tax, omitting the month prior to the tax. \FEnote \Regnote"


esttab m* using $tables/`name'.tex, cells(b(fmt(3) star) se(fmt(3) par)) ///
    s(treatedhh households household_months balance weights) ///
    drop(_cons) nobase label ///
    mlabels(none) collabels(none) booktabs replace substitute(\_ _) ///
    prehead("\begin{spacing}{1.0} \begin{table} \centering \caption{`caption'} \label{`name'} \begin{threeparttable} \begin{tabular}{m{0.23\linewidth}*{@E}{>{\centering\arraybackslash}m{0.10\linewidth}}} \toprule") ///
    postfoot("\bottomrule \end{tabular} \Fignote{`note'} \end{threeparttable} \end{table} \end{spacing}" )

restore

### Using other cities as controls

Cook DMA may be a good control if one wants to separate effects of the tax from effects of information effects from media coverage of the tax and changing opinions about the healthfulness of sugar. However, grocers in untaxed areas in the Chicago DMA may have responded to the tax by increasing advertising, lowering prices, offering coupons, etc., which could contaminate the experiment. As such, here we use other cities that did not pass the tax as a control group.

In [ ]:
* get data
use $gen_data/panelist_nutrition_month_prepped.dta, clear
top_code_all


* gen indicators for different controls
replace evertreated = 0 if dma_cd != 602

* SF and Seattle as controls 
gen control1 = inlist(dma_cd, 819, 807) & inlist(locality, "Control", "San Francisco", "Seattle")

* those inside Chicago DMA not inside Cook
gen control2 = dma_cd == 602 & locality == "Control"

* top 10 largest DMAs that did NOT pass a tax
* NYC, LA, DFW, ATL, BOS, Tampa, HOU, MIN, DC, DET
gen control3 = inlist(dma_cd, 501, 803, 623, 524, 506, 539, 618, 613, 511, 505)


* keep relevant obs
keep if evertreated + control1 + control2 + control3 > 0

* add months_since_treated for control units
replace months_since_treat = . if dma_cd != 602
bys yearmonth: ereplace months_since_treat = min(months_since_treat)

* drop last month (incomplete data)
drop if months_since_treat == 28

* import demographic data
merge m:1 household_code year using $gen_data/panelist_demographics.dta, keep(3) nogen

In [ ]:
* def CEM function

capture program drop get_cem
program get_cem
    capture drop cem*

    * mark one obs per person to match
    bys household_code year: gen temp = ( _n == 1 & year == 2017)

    * match obs
    cem hh_race (1 2 3 4) ///
        hh_size (1 2 3 4 5 6) ///
        hh_income (1 2 3 4) ///
        hh_age (1 2 3 4) ///
        hh_educ (1 2 3 4) ///
        hh_child (0 1) ///
        hh_emp_male (0 1 2 3 9) ///
        hh_emp_female (0 1 2 3 9) if temp == 1, tr(evertreated)
    bys household_code: ereplace cem_matched = min(cem_matched)
    drop temp
    end

In [ ]:
* plot raw


* Nutrients

capture restore
preserve

* trip sample and apply cem
keep if evertreated | control3
get_cem
keep if cem_matched

* percent vars
gen p_sugar = (sugargrams * 4) / calories

local vars = "calories-sugargrams p_sugar"

collapse (mean) `vars', by(evertreated months_since_treat)

// args var title plotname ytitle
plot_cook calories "Calories per month" "calories" "kcal/month"
plot_cook sugargrams "Sugars per month" "sugar" "g/month"
plot_cook p_sugar "Percent calories from sugar" "p_sugar" "Percent"
plot_cook fatgrams "Fats per month" "fats" "g/month"
plot_cook satfatgrams "Saturated fats per month" "satfats" "g/month"
plot_cook carbsgrams "Carbohydrates per month" "carbs" "g/month"
plot_cook fibergrams "Fiber per month" "fiber"  "g/month"
plot_cook proteingrams "Protein per month" "protein" "g/month"
plot_cook sodiumgrams "Sodium per month" "sodium" "g/month"

restore

In [ ]:
* Define function to estimate avg effect

capture program drop reg_nutr_avg
program reg_nutr_avg
    args nutr keep_cond

    capture restore
    preserve

    * keep if
    keep if `keep_cond'

    * gen indicators for treatment months
    capture gen indic_pretax = inrange(months_since_treat, -4, -1)
    capture gen indic_tax = inrange(months_since_treat, 0, 3)
    capture gen indic_posttax = inrange(months_since_treat, 4, 7)

    * run regression
    capture drop log_`nutr'
    gen log_`nutr' = asinh(`nutr')
    reghdfe log_`nutr' c.evertreated#c.indic_pretax ///
        c.evertreated#c.indic_tax c.evertreated#c.indic_posttax ///
        [fw = projection_factor], ///
        absorb(household_code months_since_treat) vce(cluster panelist_zipcd)
    drop log_`nutr'

    restore

    end

In [ ]:
* fit for different nutrients and counterfactuals

eststo clear
local control = (dma_cd == 602 & locality == `""Control""' )

reg_nutr_avg sugargrams "inrange(months_since_treat, -12, 7) & dma_cd == 602"
eststo m1
reg_nutr_avg sugargrams "inrange(months_since_treat, -12, 7) & dma_cd == 602 & cem_matched"
eststo m2
reg_nutr_avg sugargrams "inrange(months_since_treat, -12, 7) & !`control'"
eststo m3
reg_nutr_avg sugargrams "inrange(months_since_treat, -12, 7) & !`control' & cem_matched"
eststo m4

esttab m*

## Additional plots

### Cook County map

In [ ]:
* Run shp2dta to get Stata recognizable data

shp2dta using $raw_data/maps/zip.shp, ///
    database($raw_data/maps/zipdb) ///
    coordinates($raw_data/maps/zipcoord) ///
    genid(zipid) replace

shp2dta using $raw_data/maps/county.shp, ///
    database($raw_data/maps/cntydb) ///
    coordinates($raw_data/maps/cntycoord) ///
    genid(cntyid) replace

In [ ]:
*** Zip-code layer plot

capture restore

* Load coordinate data
use $raw_data/maps/zipdb, clear
rename *, lower
gen zip = real(zcta)

* keep only Chicago DMA, and merge border data
merge 1:1 zip using $raw_data/maps/zip_list, keep(3) nogen
merge 1:1 zip using $raw_data/maps/border, keep(3) nogen

* Merge coefficient data
preserve
use $raw_data/maps/coeff_cook_tot_border.dta, clear  // TODO: update this
gen border = real(substr(var, 1, 1))
keep border coef
drop if border == .
bys coef: gen n = 1 if _n == 1
replace coef = sum(n)
rename coef coeff_tot
tempfile temp
qui save `temp'
restore
merge m:1 border using `temp', nogen


** Create Cook Co Boundary
* Load county coordinate data 
preserve

use $raw_data/maps/cntydb, clear
rename *, lower
gen countyfip = real(geoid)
keep if countyfip == 17031
save $raw_data/maps/cntydb2, replace

mergepoly cntyid using $raw_data/maps/cntycoord, ///
    coord($raw_data/maps/cookcoord) replace

restore

* keep relevant lats and longs
gen lat = real(intptlat)
gen lon = real(intptlon)
keep if inrange(lat, 41.2, 42.4)
keep if inrange(lon, -88.4, -87.2)

* fix colors
* 1 layer 2
* 2 layer 3
* 3 layer 1
* 4 = outside cook
* 5 = border
* 6 = cross border
* 7 = innermost
replace coeff_tot = 2 if coeff_tot == 7
replace coeff_tot = 3 if coeff_tot == 1

* Create map using coeff as intensity var
spmap coeff_tot using $raw_data/maps/zipcoord, ///
    id(zipid) ndfcolor(white) clnumber(5) fcolor(RdYlBu) clm(k) ///
    polygon(data($raw_data/maps/cookcoord) fcolor(none) ocolor(yellow) osize(thick)) ///
    name(cookzips, replace) mfcolor(black) ///
    legend(off)
//    title("Cook County layers by zip code")

export_img cookzips

### Google Trends

Examine Google Trends data to have some sense of salience and anticipation effects.

In [ ]:
* import and prep data

import delimited $raw_data/gtrends.csv, varnames(1) clear delimiters(",")
rename sodatax sodatax
gen date = date(week, "MDY")
gen year = year(date)
gen week2 = week(date)
gen month = month(date)

gen yearweek = week2
replace yearweek = yearweek+52 if year == 2017

local passed = week(date("11/10/2016", "MDY"))
local delayed = week(date("6/27/2017", "MDY")) + 52
local start = week(date("8/2/2017", "MDY")) + 52
local delayed2 = week(date("9/13/2017", "MDY")) + 52
local repeal = week(date("10/11/2017", "MDY")) + 52
local end = week(date("12/1/2017", "MDY")) + 52

capture drop label
gen label = ""
replace label = "Passed" if yearweek == `passed'
replace label = "Tax Delayed" if yearweek == `delayed'
replace label = "Start" if yearweek == `start'
replace label = "Vote" if yearweek == `delayed2'
replace label = "Delayed" if yearweek == `delayed2' + 1
replace label = "Repealed" if yearweek == `repeal'
replace label = "End" if yearweek == `end'

capture drop marker*
gen markerx = yearweek - 3
gen markery = sodatax + 4
replace markerx = markerx - 3 if label == "Tax Delayed"
replace markerx = markerx - 1.5 if label == "Start"
replace markerx = markerx - 1 if label == "Repealed"
replace markerx = markerx + 3 if label == "End"
replace markerx = markerx - 1.5 if label == "Delayed"
replace markery = markery + 32 if label == "Delayed"
replace markery = markery + 4 if label == "Vote"
replace markerx = markerx + 1 if label == "Vote"
replace markery = markery - 3 if inlist(label, "Start", "End")


twoway (scatter markery markerx, mlabel(label) mlabcol(black) msize(0) mcol(%0)) ///
    (line sodatax yearweek, lcol(navy) xline(83 100, lpattern(dash))), ///
    xlabel(45 `" "Nov 10" "2016" "' 78 `" "June 27" "2017" "' ///
            93 `" "Oct 11" "2017" "' 100 `" "Dec 1" "2017" "') ///
    legend(off) ytitle("Relative Volume") ///
    name(gtrends, replace)
//    title(`"Relative Weekly Search Volume for "soda tax" "') ///

export_img gtrends